<a href="https://colab.research.google.com/github/mukhammadsiddiq/working_with_CSV/blob/main/motionDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: !pip install kaggle

!pip install --upgrade --force-reinstall --no-deps kaggle


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105119 sha256=0501c4f68391bbb58dad4075b2b49762e736dddd93938477bb1c1666ecd4192d
  Stored in directory: /root/.cache/pip/wheels/d7/54/06/8a8f40cb39536605feb9acaacd0237a95eba39e5065e6392f4
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.14
    Uninstalling kaggle-1.6.14:
      Successfully uninstalled kaggle-1.6.14


In [3]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d fdraeger/frontdoormotionbrightness


Saving kaggle.json to kaggle (1).json
Dataset URL: https://www.kaggle.com/datasets/fdraeger/frontdoormotionbrightness
License(s): unknown
  0% 0.00/209k [00:00<?, ?B/s]
100% 209k/209k [00:00<00:00, 67.6MB/s]


In [4]:
!unzip frontdoormotionbrightness.zip -d /content/dataset

Archive:  frontdoormotionbrightness.zip
  inflating: /content/dataset/brightness_2017-5-26.csv  
  inflating: /content/dataset/contacts_2017-5-26.csv  
  inflating: /content/dataset/motion_2017-5-26.csv  


In [5]:
# Step 1: Install Kaggle API
!pip install kaggle


In [53]:
import numpy as np
import pandas as pd
import glob  # For handling file paths with wildcards
import zipfile  # For handling zip files
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.preprocessing import MinMaxScaler  # For normalizing data



In [22]:
with zipfile.ZipFile('frontdoormotionbrightness.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


In [52]:
csv_files = glob.glob('dataset/*.csv')

for file in csv_files:
    data = pd.read_csv(file)
    print(data.head())

# Step 3: Preprocess the data
data = data.drop_duplicates()
data = data.dropna()

# Extract numerical features for scaling
numerical_features = data.select_dtypes(include=['number']).columns
data_to_scale = data[numerical_features]

# Step 4: Scale the numerical data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_to_scale)

# Convert scaled data back to DataFrame
data_scaled = pd.DataFrame(data_scaled, columns=numerical_features)

# Concatenate scaled numerical features with non-numerical features
data_processed = pd.concat([data.drop(numerical_features, axis=1), data_scaled], axis=1)

# Print the columns of the processed DataFrame to identify the correct target variable column
print(data_processed.columns)

# Step 5: Split the data into training and testing sets
X = data_processed.drop('isDetected', axis=1).values
y = data_processed['isDetected'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 6: Reshape the data for LSTM input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# ... (Rest of the code remains the same)
# Step 7: Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(LSTM(50))
model_lstm.add(Dense(1))

# Step 8: Compile the model
model_lstm.compile(optimizer='adam', loss='mse')


   Unnamed: 0            timestamp      contact  isClosed  doy  dow  year  \
0           0  2017-05-01 01:47:53   _Main_Door     False  121    0  2017   
1           1  2017-05-01 01:47:58   _Main_Door      True  121    0  2017   
2           4  2017-05-01 01:58:19   _Main_Door     False  121    0  2017   
3           8  2017-05-01 01:58:41   _Main_Door      True  121    0  2017   
4          11  2017-05-01 02:10:22  _SZ_Terasse      True  121    0  2017   

     tod  
0  01:47  
1  01:47  
2  01:58  
3  01:58  
4  02:10  
   Unnamed: 0            timestamp  brightness  doy  dow  year    tod
0           0  2016-12-20 09:42:55       193.0  355    1  2016  09:42
1           1  2016-12-20 09:49:16       194.0  355    1  2016  09:49
2           2  2016-12-20 09:51:34       194.0  355    1  2016  09:51
3           3  2016-12-20 09:53:52       193.0  355    1  2016  09:53
4           4  2016-12-20 09:56:10       193.0  355    1  2016  09:56
   Unnamed: 0            timestamp  isDetected  doy